In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
%matplotlib inline
df = pd.read_csv('Airbnb_data/listings_SanFan.csv')
print(list(df))
# u'zipcode',u'location_price',,u'instant_bookable', u'host_is_superhost',u'host_response_rate',
selected_features = [u'price',u'accommodates',u'host_response_time',
       u'bathrooms', u'bedrooms', u'beds',u'security_deposit', u'cleaning_fee', u'guests_included',
       u'extra_people', u'minimum_nights', u'maximum_nights',u'guests_included',  
       u'availability_365','latitude','longitude', 'zipcode',u'location_price',u'instant_bookable', u'host_is_superhost',u'host_response_rate',
       u'number_of_reviews', u'review_scores_rating',u'review_scores_cleanliness', u'review_scores_checkin',
       u'review_scores_communication', u'review_scores_location',
       u'review_scores_value', u'house_rules',u'amenities','bed_type', 'room_type', 'cancellation_policy', 'property_type']
df = df.loc[:, selected_features]
# df.apply(lambda x:x.fillna(x.value_counts().index[0], inplace=True))
# df.fillna(method = 'backfill', inplace = True)
# df.dropna(inplace=True)

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_price', '

In [27]:
# df['review_scores_rating'].describe()
df['zipcode'].unique()
# def clean_zipcode(row):
#     return row[:7]
# df['zipcode'] = df.apply(clean_zipcode)
# df['zipcode'].unique()

# pd.to_numeric(df['zipcode'], errors = 'coerce')
# df['zipcode'][df['zipcode'] == '60660-1448']
# df['zipcode'] = df['zipcode'].apply(lambda x: str(x)[:6])
# df['zipcode'].unique()
# df['zipcode'].astype(float)
# sns.distplot(df['zipcode'])
# len(df)

array(['60625', '60640', nan, '60613', '60618', '60659', '60615', '60637',
       '60626', '60645', '60660', '60630', '60646', '60634', '60647',
       '60622', '60614', '60642', '60612', '60654', '60661', '60651',
       '60639', '60707', '60302', '60644', '60304', '60607', '60608',
       '60606', '60616', '60605', '60633', '60656', '60631', '60610',
       '60611', '60602', '60601', '60657', '60603', '60649', '60619',
       '60617', '60827', '60638', '60609', '60636', '60621', '60652',
       '60453', '60660-1448', '60653', '60632', '60641', '60624', '60623',
       '60628', '60411', '60629', '60620', '60643', '60202', '60805'],
      dtype=object)

In [19]:
import pysal
from pysal.cg.kdtree import KDTree    

locations = [(40.702566, -73.816859),
         (40.70546, -73.810708),
         (40.709179, -73.820574),
         (40.700486, -73.807969),
         (40.694624, -73.820593),
         (40.695132, -73.820841),
         (40.694095, -73.821334),
         (40.694165, -73.822368),
         (40.695077, -73.822817),
         (40.6747769261, -73.8092618174)] 
tree = KDTree(locations, distance_metric='Arc', radius=pysal.cg.RADIUS_EARTH_MILES)
current_point = (40.709523, -73.802472)
# get all points within 1 mile of 'current_point'
indices = tree.query_ball_point(current_point, 30)
for i in indices:
    print(locations[i])


ModuleNotFoundError: No module named 'pysal'

In [4]:
# FEES and PRICES
df['price'] = df['price'].str.replace("\$|,", "").astype(float)
df['security_deposit'] = df['security_deposit'].str.replace("\$|,", "").astype(float)
df['cleaning_fee'] = df['cleaning_fee'].str.replace("\$|,", "").astype(float)
df['extra_people'] = df['extra_people'].str.replace("\$|,", "").astype(float)
df['availability'] = df['availability_365'] / 365

In [5]:
# remove rows that have 'NaN'in key features
# solve NaN cells in unimportant attributes

# remove_criteria = df['price'].isnull() | df['zipcode'].isnull()
# df = df[-remove_criteria]

In [6]:
# HOUSE RULES
house_rules = df['house_rules'].str.lower()

smoking = house_rules.str.contains("smoke|smoking", na= False)
df.loc[:, 'smoking'] = - smoking # False: No smoking allowed

pet = house_rules.str.contains("pet", na=False)
df.loc[:, 'pet'] = - pet

party = house_rules.str.contains("party|parties", na=False)
df.loc[:, 'party'] = - party

guest = house_rules.str.contains("guest|guests", na=False)
df.loc[:, 'guest'] = - guest

df = df.drop(['house_rules'], axis = 1)


In [7]:
import re
# AMENITIES
amenities = list(df['amenities'])
total = ','.join(amenities)
total = total.replace("{", "").replace("}","").replace("\"", "").split(",")
amenity_items = list(set(total))
amenity_items = list(filter(None, amenity_items))
for item in amenity_items:
    if re.match(r'translation',item):
        amenity_items.remove(item)

In [8]:
# Turn Amenities into OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

amenities = list(df['amenities'])
# print(amenity_items)
# amenities
# df['amenities'].value_counts()
new_table = pd.DataFrame(index = df.reset_index().values[:,0], columns = amenity_items).fillna(0)
# new_table.head()
for i in range(len(amenities)):
    for item in amenity_items:
        if item in amenities[i]:
            new_table.set_value(i, item, 1)
sum_table = np.array(new_table.sum())
ind = (-sum_table).argsort()[:60]
sort_table = new_table.sum().iloc[ind]
df = df.drop(['amenities'], axis = 1)
df = pd.concat([df, sort_table], axis = 1)

/Users/tjhuynh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
/Users/tjhuynh/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [9]:
# reduce list of amenities

# fill NaN with most frequent values
df.head()

,price,accommodates,host_response_time,bathrooms,bedrooms,beds,security_deposit,cleaning_fee,guests_included,extra_people,...,bed_type,room_type,cancellation_policy,property_type,availability,smoking,pet,party,guest,0
0,56.0,4.0,within an hour,1.0,1.0,2.0,150.0,30.0,2.0,15.0,...,Real Bed,Private room,moderate,House,0.087671,True,True,True,True,NaN
1,36.0,2.0,within an hour,1.0,1.0,1.0,150.0,35.0,1.0,20.0,...,Real Bed,Private room,strict,Condominium,0.000000,False,False,True,False,NaN
2,80.0,6.0,within an hour,1.5,2.0,3.0,150.0,65.0,4.0,15.0,...,Real Bed,Private room,moderate,Townhouse,0.432877,True,True,True,True,NaN
3,80.0,2.0,within a few hours,1.0,1.0,1.0,200.0,10.0,2.0,10.0,...,Real Bed,Private room,flexible,Condominium,0.142466,True,True,True,False,NaN
4,20.0,3.0,within an hour,1.0,1.0,1.0,200.0,35.0,1.0,10.0,...,Real Bed,Private room,strict,Apartment,0.315068,False,True,True,False,NaN


In [10]:
pd.isnull(df['host_response_time'].iloc[5204])
column_list = list(df['host_response_time'])
len(column_list)

5263

In [11]:
unique_values =list(df['host_response_time'].unique())[:-1]
unique_values
column_list = list(df['host_response_time'])
new_table = pd.DataFrame(index = df.reset_index().values[:,0], columns = unique_values).fillna(0)
new_table.head()
for i in range(len( df )):
        for item in unique_values:
            print((item,i))
            if item in column_list[i] and pd.isnull(column_list[i])==False:
                new_table.set_value(i, item, 1) 

/Users/tjhuynh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


('within an hour', 0)
('within a few hours', 0)
('within a day', 0)
('a few days or more', 0)
('within an hour', 1)
('within a few hours', 1)
('within a day', 1)
('a few days or more', 1)
('within an hour', 2)
('within a few hours', 2)
('within a day', 2)
('a few days or more', 2)
('within an hour', 3)
('within a few hours', 3)
('within a day', 3)
('a few days or more', 3)
('within an hour', 4)
('within a few hours', 4)
('within a day', 4)
('a few days or more', 4)
('within an hour', 5)
('within a few hours', 5)
('within a day', 5)
('a few days or more', 5)
('within an hour', 6)
('within a few hours', 6)
('within a day', 6)
('a few days or more', 6)
('within an hour', 7)
('within a few hours', 7)
('within a day', 7)
('a few days or more', 7)
('within an hour', 8)
('within a few hours', 8)
('within a day', 8)
('a few days or more', 8)
('within an hour', 9)
('within a few hours', 9)
('within a day', 9)
('a few days or more', 9)
('within an hour', 10)
('within a few hours', 10)
('within a

('within an hour', 480)
('within a few hours', 480)
('within a day', 480)
('a few days or more', 480)
('within an hour', 481)
('within a few hours', 481)
('within a day', 481)
('a few days or more', 481)
('within an hour', 482)
('within a few hours', 482)
('within a day', 482)
('a few days or more', 482)
('within an hour', 483)
('within a few hours', 483)
('within a day', 483)
('a few days or more', 483)
('within an hour', 484)
('within a few hours', 484)
('within a day', 484)
('a few days or more', 484)
('within an hour', 485)
('within a few hours', 485)
('within a day', 485)
('a few days or more', 485)
('within an hour', 486)
('within a few hours', 486)
('within a day', 486)
('a few days or more', 486)
('within an hour', 487)
('within a few hours', 487)
('within a day', 487)
('a few days or more', 487)
('within an hour', 488)
('within a few hours', 488)
('within a day', 488)
('a few days or more', 488)
('within an hour', 489)
('within a few hours', 489)
('within a day', 489)
('a few 

('a few days or more', 854)
('within an hour', 855)
('within a few hours', 855)
('within a day', 855)
('a few days or more', 855)
('within an hour', 856)
('within a few hours', 856)
('within a day', 856)
('a few days or more', 856)
('within an hour', 857)
('within a few hours', 857)
('within a day', 857)
('a few days or more', 857)
('within an hour', 858)
('within a few hours', 858)
('within a day', 858)
('a few days or more', 858)
('within an hour', 859)
('within a few hours', 859)
('within a day', 859)
('a few days or more', 859)
('within an hour', 860)
('within a few hours', 860)
('within a day', 860)
('a few days or more', 860)
('within an hour', 861)
('within a few hours', 861)
('within a day', 861)
('a few days or more', 861)
('within an hour', 862)
('within a few hours', 862)
('within a day', 862)
('a few days or more', 862)
('within an hour', 863)
('within a few hours', 863)
('within a day', 863)
('a few days or more', 863)
('within an hour', 864)
('within a few hours', 864)
('

('a few days or more', 1354)
('within an hour', 1355)
('within a few hours', 1355)
('within a day', 1355)
('a few days or more', 1355)
('within an hour', 1356)
('within a few hours', 1356)
('within a day', 1356)
('a few days or more', 1356)
('within an hour', 1357)
('within a few hours', 1357)
('within a day', 1357)
('a few days or more', 1357)
('within an hour', 1358)
('within a few hours', 1358)
('within a day', 1358)
('a few days or more', 1358)
('within an hour', 1359)
('within a few hours', 1359)
('within a day', 1359)
('a few days or more', 1359)
('within an hour', 1360)
('within a few hours', 1360)
('within a day', 1360)
('a few days or more', 1360)
('within an hour', 1361)
('within a few hours', 1361)
('within a day', 1361)
('a few days or more', 1361)
('within an hour', 1362)
('within a few hours', 1362)
('within a day', 1362)
('a few days or more', 1362)
('within an hour', 1363)
('within a few hours', 1363)
('within a day', 1363)
('a few days or more', 1363)
('within an hour'

...

('within an hour', 3729)
('within a few hours', 3729)
('within a day', 3729)
('a few days or more', 3729)
('within an hour', 3730)
('within a few hours', 3730)
('within a day', 3730)
('a few days or more', 3730)
('within an hour', 3731)
('within a few hours', 3731)
('within a day', 3731)
('a few days or more', 3731)
('within an hour', 3732)
('within a few hours', 3732)
('within a day', 3732)
('a few days or more', 3732)
('within an hour', 3733)
('within a few hours', 3733)
('within a day', 3733)
('a few days or more', 3733)
('within an hour', 3734)
('within a few hours', 3734)
('within a day', 3734)
('a few days or more', 3734)
('within an hour', 3735)
('within a few hours', 3735)
('within a day', 3735)
('a few days or more', 3735)
('within an hour', 3736)
('within a few hours', 3736)
('within a day', 3736)
('a few days or more', 3736)
('within an hour', 3737)
('within a few hours', 3737)
('within a day', 3737)
('a few days or more', 3737)
('within an hour', 3738)
('within a few hours'

('a few days or more', 4217)
('within an hour', 4218)
('within a few hours', 4218)
('within a day', 4218)
('a few days or more', 4218)
('within an hour', 4219)
('within a few hours', 4219)
('within a day', 4219)
('a few days or more', 4219)
('within an hour', 4220)
('within a few hours', 4220)
('within a day', 4220)
('a few days or more', 4220)
('within an hour', 4221)
('within a few hours', 4221)
('within a day', 4221)
('a few days or more', 4221)
('within an hour', 4222)
('within a few hours', 4222)
('within a day', 4222)
('a few days or more', 4222)
('within an hour', 4223)
('within a few hours', 4223)
('within a day', 4223)
('a few days or more', 4223)
('within an hour', 4224)
('within a few hours', 4224)
('within a day', 4224)
('a few days or more', 4224)
('within an hour', 4225)
('within a few hours', 4225)
('within a day', 4225)
('a few days or more', 4225)
('within an hour', 4226)
('within a few hours', 4226)
('within a day', 4226)
('a few days or more', 4226)
('within an hour'

('a few days or more', 4603)
('within an hour', 4604)
('within a few hours', 4604)
('within a day', 4604)
('a few days or more', 4604)
('within an hour', 4605)
('within a few hours', 4605)
('within a day', 4605)
('a few days or more', 4605)
('within an hour', 4606)
('within a few hours', 4606)
('within a day', 4606)
('a few days or more', 4606)
('within an hour', 4607)
('within a few hours', 4607)
('within a day', 4607)
('a few days or more', 4607)
('within an hour', 4608)
('within a few hours', 4608)
('within a day', 4608)
('a few days or more', 4608)
('within an hour', 4609)
('within a few hours', 4609)
('within a day', 4609)
('a few days or more', 4609)
('within an hour', 4610)
('within a few hours', 4610)
('within a day', 4610)
('a few days or more', 4610)
('within an hour', 4611)
('within a few hours', 4611)
('within a day', 4611)
('a few days or more', 4611)
('within an hour', 4612)
('within a few hours', 4612)
('within a day', 4612)
('a few days or more', 4612)
('within an hour'

('a few days or more', 5103)
('within an hour', 5104)
('within a few hours', 5104)
('within a day', 5104)
('a few days or more', 5104)
('within an hour', 5105)
('within a few hours', 5105)
('within a day', 5105)
('a few days or more', 5105)
('within an hour', 5106)
('within a few hours', 5106)
('within a day', 5106)
('a few days or more', 5106)
('within an hour', 5107)
('within a few hours', 5107)
('within a day', 5107)
('a few days or more', 5107)
('within an hour', 5108)
('within a few hours', 5108)
('within a day', 5108)
('a few days or more', 5108)
('within an hour', 5109)
('within a few hours', 5109)
('within a day', 5109)
('a few days or more', 5109)
('within an hour', 5110)
('within a few hours', 5110)
('within a day', 5110)
('a few days or more', 5110)
('within an hour', 5111)
('within a few hours', 5111)
('within a day', 5111)
('a few days or more', 5111)
('within an hour', 5112)
('within a few hours', 5112)
('within a day', 5112)
('a few days or more', 5112)
('within an hour'

TypeError: argument of type 'float' is not iterable

In [14]:
columns = ['bed_type', 'room_type', 'cancellation_policy', 'property_type', 'host_response_time']

for column in columns:  
    unique_values = list(df[column].unique())
    column_list = list(df[column])
    new_table = pd.DataFrame(index = df.reset_index().values[:,0], columns = unique_values).fillna(0)
    
    for i in range(len( column_list )):
        for item in unique_values:
            if item in column_list[i]:
                new_table.set_value(i, item, 1)  
    df = pd.concat( [df, new_table], axis = 1)
    df = df.drop([column], axis = 1)         
# df.columns.values

/Users/tjhuynh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()


TypeError: 'in <string>' requires string as left operand, not float

In [ ]:
# 'date'	neighborhood	Safety of neighborhood	Variable cost	Initial Investment	Accomodation	Capacity	Price per day

In [ ]:
df.dtypes
# df['cancellation_policy'].unique()

In [15]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

cluster_data=df.ix[:,2:]
X=cluster_data
y=df.ix[:,'price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

tunedParameters=[{'n_estimators':range(50,100,10)}]
clf=GridSearchCV(RandomForestRegressor(n_jobs = -1, criterion='mse'), param_grid = tunedParameters,cv=10)

clf.fit(X, y)

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

/Users/tjhuynh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':
/Users/tjhuynh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
/Users/tjhuynh/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behav

ValueError: could not convert string to float: 'within a day'

In [ ]:
X=cluster_data
y=df.ix[:,'price']
from collections import OrderedDict
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
ensemble_clfs = [("RandomForestClassifier, max_features=None",
        RandomForestClassifier(warm_start=True, max_features=None,
                               oob_score=True,
                               random_state=0))]

# Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)

# Range of `n_estimators` values to explore.
min_estimators = 10
max_estimators = 100

for label, clf in ensemble_clfs:
    for i in range(min_estimators, max_estimators + 1):
        clf.set_params(n_estimators=i)
        clf.fit(X, y)

        # Record the OOB error for each `n_estimators=i` setting.
        oob_error = 1 - clf.oob_score_
        error_rate[label].append((i, oob_error))

# Generate the "OOB error rate" vs. "n_estimators" plot.
for label, clf_err in error_rate.items():
    xs, ys = zip(*clf_err)
    plt.plot(xs, ys, label=label)

plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()

from sklearn.ensemble import RandomForestRegressor

X=cluster_data
y=df.ix[:,'price']
tunedParameters = [{'n_estimators':100}]

clf2 = RandomForestRegressor(n_jobs = 1, criterion='mse', n_estimators=100)
#Fit Model
clf2.fit(X, y)

In [ ]:
FeatImp = pd.DataFrame({'feature': list(X.columns), 'importance': list(clf2.feature_importances_)})
FeatImp = FeatImp.sort_values('importance', ascending = False)
#Set Index To Field You want to Sort Bar Chart By
FeatImp = FeatImp.set_index('feature')
FeatImp.head(100)
FeatImp.to_csv('feature_imp.csv')

In [ ]:
FeatImp.index[0:30]

In [ ]:
FeatImp['importance'].values[0:30]

In [ ]:
import matplotlib.pyplot as plt
plt.figure()

plt.title("Feature Importance")
y_pos = np.arange(len(FeatImp.index[0:30]))
plt.bar(y_pos,FeatImp['importance'].values[0:30])
plt.xticks(y_pos, FeatImp.index[0:30],rotation='vertical')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(17,10))
# df['price'] = df.price.str.replace("\$|,", "").astype(float)
# df['price'].hist()
sns.distplot(df['price'])
# plt.show()
mean_price = df.price.iloc[:5].mean()
mean_price
# df['host_acceptance_rate'].head()

In [ ]:
# df['review_scores_rating'].fillna(0, inplace=True)
df['review_scores_rating'].dropna(axis=0, inplace=True)
sns.distplot(df['review_scores_rating'])


In [ ]:
# Correlation Plot
var = 'review_scores_rating'
data = pd.concat([df['price'], df[var]], axis=1)
data.plot.scatter(x=var, y='price')
data.plot.scatter(x=var, y='price', ylim=(0,1500))

In [ ]:
# cluster by locations
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans

def plot_3D_clusters(X, k):
    """Plot 3 attributes in dataset to explore natural clusters within data"""
    estimators = {'k_means_3': KMeans(n_clusters=k)}
    fignum = 1
    for name, est in estimators.items():
        fig = plt.figure(fignum, figsize=(4, 3))
        plt.clf()
        ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
        plt.cla()
        est.fit(X)
        labels = est.labels_
        # Change the 2nd column in X[:,_] to choose attributes for plotting
        ax.scatter(X[:,0], X[:,1], X[:,2], c=labels.astype(np.float),edgecolor='k')
        ax.w_xaxis.set_ticklabels([])
        ax.w_yaxis.set_ticklabels([])
        ax.w_zaxis.set_ticklabels([])
        ax.set_xlabel('latitude')
        ax.set_ylabel('longitude')
        ax.set_zlabel('price')
        fignum = fignum + 1
    plt.show()
# change the n
df = pd.read_csv('Airbnb_data/listings_SanFan.csv')
df['price'] = df['price'].str.replace("\$|,", "").astype(float)
X = np.array(df[['latitude', 'longitude', 'price']])
plot_3D_clusters(X, 4)

In [ ]:
# Regression model
import statsmodels.api as sm # import statsmodels 

In [ ]:
df['price'].describe()
df['host_acceptance_rate'].describe()
df['host_response_rate']

In [ ]:
# Transformation
# right skewed: log transform
# left skewed: power transform


In [ ]:
# Create train and test data
train_df, test_df = 
norm_
norm_train_df

In [ ]:
from sklearn.neighbors import KNeighborRegressor
# Create KNN model: 5 closest neighbors
knn = KNeighborRegressor(algorithm='brute', n_neighbors = 5)
cols = ['accommodates', 'bedrooms', 'bathrooms', 'beds']
knn.fit(norm_train_df[col], norm_train_df['price'])
features_predictions = knn.predict(norm_test_df[cols])
features_mse = mean_squared_error(norm_test_df['price'], features_predictions)
features_rmse = features_mse ** (1/2)
features_rmse